### Import Modules

In [121]:
import numpy as np
import gvar as gv
import lsqfit
import sys
import os
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import h5py
import time

sys.path.append("../")
from fitter import data_loader as dl
from fitter import bootstrapper as bs
from fitter import fitter as fit
from fitter import special_functions as sf

for j in range(10): # Sometimes this needs to be loaded twice...
    matplotlib.rcParams['figure.figsize'] = [10, 10]

In [123]:
reload(bs)
reload(dl)
reload(fit)
reload(sf)

<module 'fitter.special_functions' from '../fitter\special_functions.pyc'>

### Specify fit parameters

In [ ]:
p_dict = {
    'order' : {
        'fit' : 'nnlo', # 'nlo', 'nnlo', or 'nnnlo'
        'vol' : 0, # max 10
        'exclude' : [], # put LECs here
        'include_log' : True,
        'include_log2' : True
    }, 
    'bs_N' : 1,  # if 0, use full list
    'fit_type' : 'ma', #'ma', 'ma-taylor, 'xpt', or 'xpt-taylor'; also 'simultaneous', 'ma-old'
    'F2' : 'FKFpi', #'FKfpi', 'FpiFpi', 'FKFK'
    'chained' : True,
    'use_boot0' : False,
    
    'use_prior' : True,
    'abbrs' : [u'a09m220', u'a09m310', u'a09m350', u'a09m400', u'a12m130',
           u'a12m220',  u'a12m220S', u'a12m220L', u'a12m310', u'a12m350',
           u'a12m400', u'a15m130', u'a15m135XL', u'a15m220', u'a15m310', u'a15m350', 'a15m400'],
    
    'make_plots' : True,
    'show_error_ellipses' : False,
    'show_bs_histograms' : False,
    
    'save_prior' : False,
    'save_results' : False,
}

### Do fit

In [ ]:
t0 = time.time()

# Load data
data_loader = dl.data_loader()
fit_data = data_loader.get_fit_data()

# Get prior
prior = None
if p_dict['use_prior']:
    prior = data_loader.get_prior(p_dict['fit_type'])

# Make bootstrapper
bootstrapper = bs.bootstrapper(fit_data, prior=prior, order=p_dict['order'], F2=p_dict['F2'], chain_fits=p_dict['chained'],
                               fit_type=p_dict['fit_type'], bs_N=p_dict['bs_N'], abbrs=p_dict['abbrs'], use_boot0=p_dict['use_boot0'])

if p_dict['make_plots']:
    data_loader.save_plots(
        bootstrapper.make_plots(
            show_error_ellipses=p_dict['show_error_ellipses'], 
            show_bootstrap_histograms=p_dict['show_bs_histograms']),
        output_filename='fits/'+bootstrapper.fit_type+'_'+bootstrapper.order['fit']
    )
else:
    print bootstrapper
    
if p_dict['save_prior']:
    print 'Saving prior...'
    data_loader.save_prior(bootstrapper.create_prior_from_fit(), p_dict['fit_type'])
    
if p_dict['save_results']:
    data_loader.save_fit_info(bootstrapper.get_fit_info())
    
    
t1 = time.time()

print "\nTotal time (s): ", t1 - t0, "\n"

Making fits...
100% complete Time (s):  2.10300016403
Compiling results...


### Or make all fits

In [44]:
p_dict = {
    'order' : {
        'vol' : 5, # max 10
        'exclude' : [],
        'include_log2' : True
    }, 
    'bs_N' : 1,  # if 0, use full list
    'F2' : 'FKFpi', #'FKfpi', 'FpiFpi', 'FKFK'
    'chained' : True,

    'use_prior' : True,
    'abbrs' : [u'a09m220', u'a09m310', u'a09m350', u'a09m400', u'a12m130',
           u'a12m220',  u'a12m220S', u'a12m220L', u'a12m310', u'a12m350',
           u'a12m400', u'a15m130', u'a15m135XL', u'a15m220', u'a15m310', u'a15m350', 'a15m400'],

    'make_plots' : True,
    'show_error_ellipses' : False,
    'show_bs_histograms' : False,

    'save_prior' : False,
    'save_results' : True,
}

In [45]:
for fit_type in ['xpt', 'xpt-taylor', 'ma', 'ma-taylor']:#, 'ma-old']:
    for order_fit in ['nlo', 'nnlo']:#, 'nnnlo']:
        
        print fit_type, order_fit
        
        # Specify fit parameters
        p_dict['order']['fit'] = order_fit
        p_dict['fit_type'] = fit_type
        
        # Do fit
        t0 = time.time()

        # Load data
        data_loader = dl.data_loader()
        fit_data = data_loader.get_fit_data()

        # Get prior
        prior = None
        if p_dict['use_prior']:
            prior = data_loader.get_prior(p_dict['fit_type'])

        # Make bootstrapper
        bootstrapper = bs.bootstrapper(fit_data, prior=prior, order=p_dict['order'], F2=p_dict['F2'], chain_fits=p_dict['chained'],
                                       fit_type=p_dict['fit_type'], bs_N=p_dict['bs_N'], abbrs=p_dict['abbrs'])

        if p_dict['make_plots']:
            data_loader.save_plots(
                bootstrapper.make_plots(
                    show_error_ellipses=p_dict['show_error_ellipses'], 
                    show_bootstrap_histograms=p_dict['show_bs_histograms']),
                output_filename='fits/'+bootstrapper.fit_type+'_'+bootstrapper.order['fit']
            )
        else:
            print bootstrapper

        if p_dict['save_prior']:
            print 'Saving prior...'
            data_loader.save_prior(bootstrapper.create_prior_from_fit(), p_dict['fit_type'])

        if p_dict['save_results']:
            data_loader.save_fit_info(bootstrapper.get_fit_info())


        t1 = time.time()
        
        #print test(bootstrapper)

        print "\nTotal time (s): ", t1 - t0, "\n"

xpt nlo
Making fits...
100% complete Time (s):  0.463999986649
Compiling results...


../fitter\special_functions.py:10: RuntimeWarning: invalid value encountered in double_scalars
  ysdev = np.abs(ss.kvp(n, gv.mean(g), 1)) * gv.sdev(g)


Done.
Saving...
Done.

Total time (s):  11.7779998779 

xpt nnlo
Making fits...
100% complete Time (s):  0.979000091553
Compiling results...
Done.
Saving...
Done.

Total time (s):  12.881000042 

xpt-taylor nlo
Making fits...
100% complete Time (s):  0.468999862671
Compiling results...
Done.
Saving...
Done.

Total time (s):  8.63800001144 

xpt-taylor nnlo
Making fits...
100% complete Time (s):  0.984000205994
Compiling results...
Done.
Saving...
Done.

Total time (s):  9.89199995995 

ma nlo
Making fits...
100% complete Time (s):  0.991000175476
Compiling results...
Done.
Saving...
Done.

Total time (s):  54.1529998779 

ma nnlo
Making fits...
100% complete Time (s):  2.49399995804
Compiling results...
Done.
Saving...
Done.

Total time (s):  56.1860001087 

ma-taylor nlo
Making fits...
100% complete Time (s):  1.11199998856
Compiling results...
Done.
Saving...
Done.

Total time (s):  49.6940000057 

ma-taylor nnlo
Making fits...
100% complete Time (s):  2.46700000763
Compiling results

In [38]:
#bootstrapper.fits[0].show_plots(view='std')